# Automomous Agent

In this notebook, we will show a simple example of and agent using the "add_tools" method of the "Agent" class. This method allows the agent to add tools to its own  inventory of tools. The agent can then use these tools to interact with the environment. 

IMPORTANT: Please note that this feature poses a security risk. The agent can use the tools to interact with the environment in ways that are not intended by the environment creator. Use it at your own risk.

In [1]:
import json
import asyncio
from agente.core.base import BaseAgent,BaseTaskAgent
from dotenv import load_dotenv
load_dotenv()

True

We start by creating an agent from the BaseAgent, but this time we set the `can_add_tools` to True. This will allow the agent to add tools to its inventory. Besides that, we will also reinforce the agent ability to add tools in the system prompt.

In [2]:
class MainAgent(BaseAgent):
    agent_name:str = "autonomous_agent"
    system_prompt:str = """You are an AI assistant and your task is to help the user.

You have available tools to help you in your task and you should use them to help the user when necessary.

You have access to a special tool called "add_tool". This tool adds new tools to your toolbox.

You should use "add_tool" tool when you need perform a task that you don't have the necessary tools.
"""

    completion_kwargs: dict = {
        "model": "claude-sonnet-4-20250514", # atention that models like gpt-4.1 might not work sometimes
        "stream": False,
    }
    can_add_tools:bool = True


In [3]:
auto_agent = MainAgent()
auto_agent.add_message(role = "user",content= """Hello. I need help with the pi number. I want to know the first 50 digits of pi. 
                       Add a tool to calculate it for me and them show me the result.""")

In [4]:
auto_agent.tools_schema

[{'type': 'function',
  'function': {'name': 'add_tool',
   'description': 'Call this tool to add a new tool (a python function) to the main list of available tools. Example: function_name = "calculate_circle_area"\nfunction_code = \'\'\'\ndef calculate_circle_area(self, radius: float) -> float:\n    """Calculate the area of a circle given its radius.     Args:\n        radius: The radius of the circle (must be a positive number).\n    """\n    import math\n    return math.pi * radius ** 2\n\'\'\'',
   'parameters': {'properties': {'function_name': {'description': 'The name of the function to be added to the list of tools.',
      'title': 'Function Name',
      'type': 'string'},
     'function_code': {'description': 'The python code of the function to be added to the list of tools, with a docstring',
      'title': 'Function Code',
      'type': 'string'}},
    'required': ['function_name', 'function_code'],
    'type': 'object'}}}]

In [5]:
response = await auto_agent.run()

Executing agent: autonomous_agent
Executing tool: add_tool (agent: autonomous_agent)
Tool get_pi_digits added successfully.
Executing agent: autonomous_agent
Executing tool: get_pi_digits (agent: autonomous_agent)
Executing agent: autonomous_agent


In [6]:
auto_agent.tools_schema

[{'type': 'function',
  'function': {'name': 'add_tool',
   'description': 'Call this tool to add a new tool (a python function) to the main list of available tools. Example: function_name = "calculate_circle_area"\nfunction_code = \'\'\'\ndef calculate_circle_area(self, radius: float) -> float:\n    """Calculate the area of a circle given its radius.     Args:\n        radius: The radius of the circle (must be a positive number).\n    """\n    import math\n    return math.pi * radius ** 2\n\'\'\'',
   'parameters': {'properties': {'function_name': {'description': 'The name of the function to be added to the list of tools.',
      'title': 'Function Name',
      'type': 'string'},
     'function_code': {'description': 'The python code of the function to be added to the list of tools, with a docstring',
      'title': 'Function Code',
      'type': 'string'}},
    'required': ['function_name', 'function_code'],
    'type': 'object'}}},
 {'type': 'function',
  'function': {'name': 'get_p

Bellow we can see first the call to the `add_tools` method  that will add a tool to the agent's inventory. Then we can see the agent using the tool to interact with the environment.

In [7]:
for m in auto_agent.conv_history.messages:
    print(m)
    print("-----------------------")

role='system' agent_name='autonomous_agent' content=[Content(type='text', text='You are an AI assistant and your task is to help the user.\n\nYou have available tools to help you in your task and you should use them to help the user when necessary.\n\nYou have access to a special tool called "add_tool". This tool adds new tools to your toolbox.\n\nYou should use "add_tool" tool when you need perform a task that you don\'t have the necessary tools.\n')] tool_calls=None tool_call_id=None tool_name=None hidden=False id=None usage=None timestamp=datetime.datetime(2025, 9, 27, 14, 48, 27, 829114)
-----------------------
role='user' agent_name='autonomous_agent' content=[Content(type='text', text='Hello. I need help with the pi number. I want to know the first 50 digits of pi. \n                       Add a tool to calculate it for me and them show me the result.')] tool_calls=None tool_call_id=None tool_name=None hidden=False id=None usage=None timestamp=datetime.datetime(2025, 9, 27, 14, 4

In [15]:
print(auto_agent.conv_history.messages[-1].content[0].text)

Perfect! Here are the first 50 digits of pi:

**3.1415926535897932384626433832795028841971693993751**

This includes:
- 1 digit before the decimal point (3)
- 49 digits after the decimal point

The tool I created uses the highly efficient Machin formula to calculate pi with arbitrary precision, ensuring accurate results for any number of digits you might need in the future.


Let's see the added tool

In [16]:
auto_agent.tools_functions

{'add_tool': <function agente.core.base.BaseAgent.add_tool(self, function_name: typing.Annotated[str, 'The name of the function to be added to the list of tools.'], function_code: typing.Annotated[str, 'The python code of the function to be added to the list of tools, with a docstring']) -> str>,
 'get_pi_digits': <function __main__.get_pi_digits(self, num_digits: int) -> str>}

In [21]:
tool_call = auto_agent.conv_history.messages[3].tool_calls[0]
arguments = tool_call['function']['arguments']
arguments_json = json.loads(arguments)
function_code = arguments_json['function_code']
print(function_code)

def get_pi_digits(self, num_digits: int) -> str:
    """Calculate pi to the specified number of digits.
    
    Args:
        num_digits: The number of digits of pi to return (including the digit before the decimal point)
    
    Returns:
        A string representation of pi with the specified number of digits
    """
    from decimal import Decimal, getcontext
    
    # Set precision high enough to calculate the required digits
    # We need extra precision for intermediate calculations
    getcontext().prec = num_digits + 10
    
    # Use the Machin formula: pi/4 = 4*arctan(1/5) - arctan(1/239)
    # This is one of the most efficient series for calculating pi
    
    def arctan(x, precision):
        getcontext().prec = precision
        x = Decimal(x)
        power = x
        result = x
        i = 1
        
        while True:
            power *= -x * x
            term = power / (2 * i + 1)
            if abs(term) < Decimal(10) ** -(precision - 5):
                break


Note that this new tool is now a method available of the agent.

In [23]:
await auto_agent.get_pi_digits(10)

'3.141592653'